# Finding the Embeddings of the Frequent Patterns

It should be easy (and fast enough) to find all embeddings of a few frequent patterns in the random forest database.

To do this, we need to 

- [x] select the right patterns (e.g. those of size six)
- [x] read my canonical string format in python / transform it to json
- [x] read the json database format in python
- [ ] write a small script that traverses the random forests and finds all embeddings and outputs them somehow

## Select the Right Patterns (e.g. those of size six)


In [38]:
# pattern size in number of vertices
patternSize = 4
# here, we count the number of edges in the pattern
patternSize = patternSize - 1 

filename = 'forests/rootedFrequentTrees/adult/WithLeafEdges/RF_10_t10.patterns'
f = open(filename)

# gives us the patterns of the selected size
frequentPatterns = filter(lambda line: line.count('(') == patternSize, f)

# gives us only the canonical strings of the patterns
cStrings = map(lambda fp: fp.split('\t')[2], frequentPatterns)

cStringList = list(cStrings)
print(cStringList)

f.close()

['61 ( leftChild 63 ( leftChild leaf ) ) ( rightChild leaf ) \n', '63 ( leftChild 0 ( leftChild leaf ) ( rightChild leaf ) ) \n', '63 ( leftChild 0 ( leftChild leaf ) ) ( rightChild leaf ) \n', '63 ( rightChild 9 ( leftChild leaf ) ( rightChild leaf ) ) \n', '63 ( rightChild 0 ( leftChild leaf ) ( rightChild leaf ) ) \n', '26 ( rightChild 9 ( leftChild leaf ) ( rightChild leaf ) ) \n', '0 ( leftChild 9 ( leftChild leaf ) ( rightChild leaf ) ) \n', '0 ( leftChild 9 ( leftChild leaf ) ) ( rightChild leaf ) \n', '0 ( rightChild 0 ( leftChild leaf ) ( rightChild leaf ) ) \n', '0 ( rightChild 63 ( leftChild leaf ) ( rightChild leaf ) ) \n', '0 ( rightChild 9 ( leftChild leaf ) ( rightChild leaf ) ) \n', '9 ( leftChild leaf ) ( rightChild 0 ( rightChild leaf ) ) \n', '9 ( rightChild 0 ( leftChild leaf ) ( rightChild leaf ) ) \n', '9 ( rightChild 9 ( leftChild leaf ) ( rightChild leaf ) ) \n']


## transform my canonical string format to json

In [44]:
s = '61 ( leftChild 63 ( leftChild leaf ) ) ( rightChild leaf ) \n'



def cString2json(cString):
    '''Pascals canonical string format and the json format used in Dortmund are 
    basically identical (up to ordering, symbols, and general feeling of course ;) ).
    This is a converter that transforms a single tree from cString format to json format 
    (entirely by string maipulation).'''
    
    intermediate = cString.replace('( leftChild', ',"leftChild":{').replace('( rightChild', ',"rightChild":{').replace(')', '}').replace('leaf', '-1 "prediction":[]')
    tokens = intermediate.split(' ')
    
    json = ''
    i = 0
    for t in tokens:
        try:
            feature = int(t)
            if feature != -1:
                s = '"id":' + str(i) + ',"feature":' + t
            else:
                s = '"id":' + str(i) + ','
            json += s
            i += 1
        except ValueError:
            json += t
            
            
    return ('{' + json.rstrip() + '}')
    
cString2json(s)

'{"id":0,"feature":61,"leftChild":{"id":1,"feature":63,"leftChild":{"id":2,"prediction":[]}},"rightChild":{"id":3,"prediction":[]}}'

In [41]:
jsons = '[' + ',\n'.join(map(cString2json, cStringList)) + ']'
print(jsons)

[{"id":0,"feature":61,"leftChild":{"id":1,"feature":63,"leftChild":{"id":2,"prediction":[]}},"rightChild":{"id":3,"prediction":[]}},
{"id":0,"feature":63,"leftChild":{"id":1,"feature":0,"leftChild":{"id":2,"prediction":[]},"rightChild":{"id":3,"prediction":[]}}},
{"id":0,"feature":63,"leftChild":{"id":1,"feature":0,"leftChild":{"id":2,"prediction":[]}},"rightChild":{"id":3,"prediction":[]}},
{"id":0,"feature":63,"rightChild":{"id":1,"feature":9,"leftChild":{"id":2,"prediction":[]},"rightChild":{"id":3,"prediction":[]}}},
{"id":0,"feature":63,"rightChild":{"id":1,"feature":0,"leftChild":{"id":2,"prediction":[]},"rightChild":{"id":3,"prediction":[]}}},
{"id":0,"feature":26,"rightChild":{"id":1,"feature":9,"leftChild":{"id":2,"prediction":[]},"rightChild":{"id":3,"prediction":[]}}},
{"id":0,"feature":0,"leftChild":{"id":1,"feature":9,"leftChild":{"id":2,"prediction":[]},"rightChild":{"id":3,"prediction":[]}}},
{"id":0,"feature":0,"leftChild":{"id":1,"feature":9,"leftChild":{"id":2,"predic

## write a small script that traverses the random forests and finds all embeddings and outputs them somehow

In [48]:
import json
import sys

j = json.loads(jsons)
print(json.dumps(j, sort_keys=True, indent=4))

[
    {
        "feature": 61,
        "id": 0,
        "leftChild": {
            "feature": 63,
            "id": 1,
            "leftChild": {
                "id": 2,
                "prediction": []
            }
        },
        "rightChild": {
            "id": 3,
            "prediction": []
        }
    },
    {
        "feature": 63,
        "id": 0,
        "leftChild": {
            "feature": 0,
            "id": 1,
            "leftChild": {
                "id": 2,
                "prediction": []
            },
            "rightChild": {
                "id": 3,
                "prediction": []
            }
        }
    },
    {
        "feature": 63,
        "id": 0,
        "leftChild": {
            "feature": 0,
            "id": 1,
            "leftChild": {
                "id": 2,
                "prediction": []
            }
        },
        "rightChild": {
            "id": 3,
            "prediction": []
        }
    },
    {
        "feature": 63,
 

There must be a smarter way than the following. 
However, this is rather easy to implement:
We iterate (recursively) over the transaction (decision) tree vertices $v$ and check whether there is a rooted subgraph isomorphism from pattern to the transaction mapping the root of pattern to $v$.
This is decided by (again) recursion over pattern and transaction simultaneously, as long as it fits.

We'll see whether this is fast enough for our case. Its something along $O(n * p)$ where $n$ and $p$ are the numbers of vertices of transactions and patterns, respectively.

In [87]:
def printInfo(pattern, transaction, where):
    print(where)
    print('p: ' + str(pattern))
    print('t: ' + str(transaction))


def recSearch(pattern, transaction, mapping):
    # check if we are in a leaf vertex in both pattern and transaction
    if 'prediction' in pattern.keys() and 'prediction' in transaction.keys():
        mapping[pattern['id']] = transaction['id']
        return True
    
    # check if we are in a split vertex in both pattern and transaction
    if 'feature' in pattern.keys() and 'feature' in transaction.keys():

        # check if split features match
        if pattern['feature'] == transaction['feature']:
            
            foundLeft = True
            foundRight = True
            if 'leftChild' in pattern.keys():
                if 'leftChild' in transaction.keys():
                    foundLeft = recSearch(pattern['leftChild'], transaction['leftChild'], mapping)    
                else:
                    foundLeft = False
                
            if 'rightChild' in pattern.keys():
                if 'rightChild' in transaction.keys():
                    foundRight = recSearch(pattern['rightChild'], transaction['rightChild'], mapping)
                else:
                    foundRight = False
                
            if foundLeft and foundRight:
                mapping[pattern['id']] = transaction['id']
                return True
            else:
                return False
            
    # if we are in the mixed case split vertex vs. leaf vertex then we cannot map the vertices on each other
    return False


def searchEmbedding(pattern, transaction):
    '''For two given root vertices, check whether pattern is a rooted 
    subtree and return a mapping id->id if so, o/w None'''
    
    mapping = dict()
    if recSearch(pattern, transaction, mapping):
        return mapping
    else:
        return None
    
    
def allEmbeddingsRec(pattern, transaction, result):
    print('process transaction vertex id ' + str(transaction['id']))
    if 'feature' in transaction.keys():
        if 'leftChild' in transaction.keys():
            allEmbeddingsRec(pattern, transaction['leftChild'], result)
        if 'rightChild' in transaction.keys():
            allEmbeddingsRec(pattern, transaction['rightChild'], result)
    
    result.append((transaction['id'], searchEmbedding(pattern, transaction)))
    return result


def allEmbeddings(pattern, transaction):
    embeddingsAndNone = allEmbeddingsRec(pattern, transaction, list())
    return embeddingsAndNone #list(filter(lambda x: x != None, embeddingsAndNone))


In [89]:
for transaction in j:
    print(allEmbeddings(j[0], transaction))

process transaction vertex id 0
process transaction vertex id 1
process transaction vertex id 2
process transaction vertex id 3
[(2, None), (1, None), (3, None), (0, {2: 2, 1: 1, 3: 3, 0: 0})]
process transaction vertex id 0
process transaction vertex id 1
process transaction vertex id 2
process transaction vertex id 3
[(2, None), (3, None), (1, None), (0, None)]
process transaction vertex id 0
process transaction vertex id 1
process transaction vertex id 2
process transaction vertex id 3
[(2, None), (1, None), (3, None), (0, None)]
process transaction vertex id 0
process transaction vertex id 1
process transaction vertex id 2
process transaction vertex id 3
[(2, None), (3, None), (1, None), (0, None)]
process transaction vertex id 0
process transaction vertex id 1
process transaction vertex id 2
process transaction vertex id 3
[(2, None), (3, None), (1, None), (0, None)]
process transaction vertex id 0
process transaction vertex id 1
process transaction vertex id 2
process transaction

In [88]:
leaf = json.loads('{"id": 1,"prediction": []}')
for transaction in j:
    print(transaction)
    print(allEmbeddings(leaf, transaction))

{'id': 0, 'feature': 61, 'leftChild': {'id': 1, 'feature': 63, 'leftChild': {'id': 2, 'prediction': []}}, 'rightChild': {'id': 3, 'prediction': []}}
process transaction vertex id 0
process transaction vertex id 1
process transaction vertex id 2
process transaction vertex id 3
[(2, {1: 2}), (1, None), (3, {1: 3}), (0, None)]
{'id': 0, 'feature': 63, 'leftChild': {'id': 1, 'feature': 0, 'leftChild': {'id': 2, 'prediction': []}, 'rightChild': {'id': 3, 'prediction': []}}}
process transaction vertex id 0
process transaction vertex id 1
process transaction vertex id 2
process transaction vertex id 3
[(2, {1: 2}), (3, {1: 3}), (1, None), (0, None)]
{'id': 0, 'feature': 63, 'leftChild': {'id': 1, 'feature': 0, 'leftChild': {'id': 2, 'prediction': []}}, 'rightChild': {'id': 3, 'prediction': []}}
process transaction vertex id 0
process transaction vertex id 1
process transaction vertex id 2
process transaction vertex id 3
[(2, {1: 2}), (1, None), (3, {1: 3}), (0, None)]
{'id': 0, 'feature': 63,

## Script, without Debug Info

In [90]:
def printInfo(pattern, transaction, where):
    print(where)
    print('p: ' + str(pattern))
    print('t: ' + str(transaction))


def recSearch(pattern, transaction, mapping):
    # check if we are in a leaf vertex in both pattern and transaction
    if 'prediction' in pattern.keys() and 'prediction' in transaction.keys():
        mapping[pattern['id']] = transaction['id']
        return True
    
    # check if we are in a split vertex in both pattern and transaction
    if 'feature' in pattern.keys() and 'feature' in transaction.keys():

        # check if split features match
        if pattern['feature'] == transaction['feature']:
            
            foundLeft = True
            foundRight = True
            if 'leftChild' in pattern.keys():
                if 'leftChild' in transaction.keys():
                    foundLeft = recSearch(pattern['leftChild'], transaction['leftChild'], mapping)    
                else:
                    foundLeft = False
                
            if 'rightChild' in pattern.keys():
                if 'rightChild' in transaction.keys():
                    foundRight = recSearch(pattern['rightChild'], transaction['rightChild'], mapping)
                else:
                    foundRight = False
                
            if foundLeft and foundRight:
                mapping[pattern['id']] = transaction['id']
                return True
            else:
                return False
            
    # if we are in the mixed case split vertex vs. leaf vertex then we cannot map the vertices on each other
    return False


def searchEmbedding(pattern, transaction):
    '''For two given root vertices, check whether pattern is a rooted 
    subtree and return a mapping id->id if so, o/w None'''
    
    mapping = dict()
    if recSearch(pattern, transaction, mapping):
        return mapping
    else:
        return None
    
    
def allEmbeddingsRec(pattern, transaction, result):
    if 'feature' in transaction.keys():
        if 'leftChild' in transaction.keys():
            allEmbeddingsRec(pattern, transaction['leftChild'], result)
        if 'rightChild' in transaction.keys():
            allEmbeddingsRec(pattern, transaction['rightChild'], result)
    
    result.append(searchEmbedding(pattern, transaction))
    return result


def allEmbeddings(pattern, transaction):
    embeddingsAndNone = allEmbeddingsRec(pattern, transaction, list())
    return list(filter(lambda x: x != None, embeddingsAndNone))



# def main(file, out):
#     f = open(file)
#     j = json.load(f)
#     f.close()

#     graphCounter = 0
#     for tree in j:
#         vertexLabels, edges = parseTree(tree)
#         transform2GraphDB(vertexLabels, edges, graphCounter, out)
#         graphCounter += 1
#     print('$')

# if __name__ == '__main__':
#     main(sys.argv[1], sys.stdout)

In [91]:
leaf = json.loads('{"id": 1,"prediction": []}')
for transaction in j:
    print(transaction)
    print(allEmbeddings(leaf, transaction))

{'id': 0, 'feature': 61, 'leftChild': {'id': 1, 'feature': 63, 'leftChild': {'id': 2, 'prediction': []}}, 'rightChild': {'id': 3, 'prediction': []}}
[{1: 2}, {1: 3}]
{'id': 0, 'feature': 63, 'leftChild': {'id': 1, 'feature': 0, 'leftChild': {'id': 2, 'prediction': []}, 'rightChild': {'id': 3, 'prediction': []}}}
[{1: 2}, {1: 3}]
{'id': 0, 'feature': 63, 'leftChild': {'id': 1, 'feature': 0, 'leftChild': {'id': 2, 'prediction': []}}, 'rightChild': {'id': 3, 'prediction': []}}
[{1: 2}, {1: 3}]
{'id': 0, 'feature': 63, 'rightChild': {'id': 1, 'feature': 9, 'leftChild': {'id': 2, 'prediction': []}, 'rightChild': {'id': 3, 'prediction': []}}}
[{1: 2}, {1: 3}]
{'id': 0, 'feature': 63, 'rightChild': {'id': 1, 'feature': 0, 'leftChild': {'id': 2, 'prediction': []}, 'rightChild': {'id': 3, 'prediction': []}}}
[{1: 2}, {1: 3}]
{'id': 0, 'feature': 26, 'rightChild': {'id': 1, 'feature': 9, 'leftChild': {'id': 2, 'prediction': []}, 'rightChild': {'id': 3, 'prediction': []}}}
[{1: 2}, {1: 3}]
{'id':

In [100]:
filename = 'forests/adult/text/RF_10.json'
f = open(filename, 'r')
transactions = json.load(f)
f.close()

patterns = j

embeddingCounts = list()

for pattern in patterns:
    counts = 0
    for transaction in transactions:
        mappings = allEmbeddings(pattern, transaction)
        counts += len(mappings)
    embeddingCounts.append(counts)
print(embeddingCounts)

[15, 17, 10, 18, 15, 12, 20, 10, 14, 12, 14, 13, 16, 23]


In [102]:
filename = 'forests/rootedFrequentTrees/adult/WithLeafEdges/RF_10_t10.patterns'
f = open(filename)

# gives us the patterns of the selected size
frequentPatterns = filter(lambda line: line.count('(') == patternSize, f)

# gives us only the canonical strings of the patterns
patternCountsFromAlg = list(map(lambda fp: int(fp.split('\t')[0]), frequentPatterns))

print(patternCountsFromAlg)

f.close()

[13, 14, 10, 15, 11, 11, 15, 10, 10, 10, 11, 10, 10, 13]


## Conclusion
We have now the tools to transform the data and compute all embeddings of frequent patterns explicitly. 
This will be soon put in a nice script that is usable from the command line.

As a fist quick glance, it seems that the patterns that were identified as frequent mostly have only one embedding per random decision tree. 
But this needs to be investigated more closely.